# Multi-view reconstruction

## 1. Prerequisites

### 1.1 Load data

In [1]:
from alinea.phenomenal.data_access.plant_1 import (
    plant_1_images_binarize,
    plant_1_calibration_camera_side,
    plant_1_calibration_camera_top)

images = plant_1_images_binarize()
calibration_side = plant_1_calibration_camera_side()
calibration_top = plant_1_calibration_camera_top()

## 2. Multi-view reconstruction

### 2.1 Associate images and projection function

In [9]:
from alinea.phenomenal.data_structure import ImageView

refs_angle_list = [120]

# Select images
image_views = list()
for angle in range(0, 360, 30):
    projection = calibration_side.get_arr_projection(angle)
    
    ref = False
    if angle in refs_angle_list:
        ref = True
    
    image_views.append(ImageView(images[angle], projection, inclusive=False, ref=ref))

projection = calibration_top.get_arr_projection(0)
image_views.append(ImageView(images[-1], projection, inclusive=True))

### 2.2 Do multi-view reconstruction

In [10]:
from alinea.phenomenal.data_structure import VoxelPointCloud
from alinea.phenomenal.multi_view_reconstruction import reconstruction_3d

voxels_size = 8
error_tolerance = 1

voxels_position = reconstruction_3d(image_views, 
                                    voxels_size=voxels_size, 
                                    error_tolerance=error_tolerance, 
                                    verbose=True)

vpc = VoxelPointCloud(voxels_position, voxels_size)

Iteration 1 / 9 : 8
Iteration 2 / 9 : 64
Iteration 3 / 9 : 512
Iteration 4 / 9 : 4096
Iteration 5 / 9 : 400
Iteration 6 / 9 : 1400
Iteration 7 / 9 : 4216
Iteration 8 / 9 : 13392
Iteration 9 / 9 : 53104


### 2.4 Viewing

In [11]:
%matplotlib notebook

from alinea.phenomenal.display import show_voxel_point_cloud

show_voxel_point_cloud(vpc,
            # size=(5000, 5000),
            color=(0.1, 0.9, 0.1),
            azimuth=310,
            distance=3000,
            elevation=90,
            focalpoint=(0, 0, 0))

# 3.Meshing

In [6]:
from alinea.phenomenal.mesh import meshing
from alinea.phenomenal.display import (
    show_voxel_point_cloud,
    show_mesh)

In [7]:
show_voxel_point_cloud(vpc,
                       # size=(5000, 5000),
                       color=(0.1, 0.9, 0.1),
                       azimuth=310,
                       distance=3000,
                       elevation=90,
                       focalpoint=(0, 0, 0))

image_3d = vpc.to_image_3d()

vertices, faces = meshing(image_3d,
                          reduction=1,
                          smoothing_iteration=10,
                          verbose=True)

print("Number of vertices : {nb_vertices}".format(nb_vertices=len(vertices)))
print("Number of faces : {nb_faces}".format(nb_faces=len(faces)))

show_mesh(vertices, faces, 
          representation="surface",
          # size=(5000, 5000),
          color=(0.1, 0.9, 0.1),
          azimuth=120,
          distance=3000,
          elevation=90,
          focalpoint=(0, 0, 0))

Marching cubes :
	Iso value : 1.0

	There are 16424 points.
	There are 33177 polygons.

Smoothing :
	Feature angle : 120.0
	Number of iteration : 10
	Pass band : 0.01

Number of vertices : 16424
Number of faces : 33177
